Rebuilding Steps, Example Network, Integrated in SNMan 'Multi' Rebuild
====================
xxx

In [1]:
import warnings
warnings.filterwarnings('ignore')

import copy
import os
import sys
import networkx as nx
import numpy as np
import shutil

import snman
from snman.constants import *
from snman import osmnx_customized as oxc
from snman import *

#PERIMETER = '_paper2_example'
#PERIMETER = '_debug'
PERIMETER = '_accessibility_debug'
#PERIMETER = 'zrh_north-west'
#PERIMETER = 'ebc_zrh_v01_10km'

data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
#export_path = os.path.join(data_directory, 'outputs', '_debug')
export_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [2]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(process_path, 'street_graph_edges.gpkg'),
    os.path.join(process_path, 'street_graph_nodes.gpkg'),
    crs=CRS_internal
)

street_graph.organize_edge_directions(G)

Load street graph


In [3]:
print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    #os.path.join(process_path, 'rebuilding_regions.gpkg'),
    os.path.join(inputs_path, 'rebuilding_regions', 'rebuilding_regions.gpkg'),
    #only_active=True,
    filter_ids=['zrh-4'],
    #filter_ids=['zrh-19'],
    #filter_ids=['zrh-main'],
    #filter_ids=['playground-1'],
    #filter_ids=['paper2_example'],
    crs=CRS_internal
)

print('Load access needs')
access_needs = snman.io.load_access_needs(
    os.path.join(inputs_path, 'switzerland', 'zurich', 'statpop', 'statpop17_zurich_city_agg_on_buildings.gpkg'),
    CRS_internal
)

access_needs['parking_spots_needed'] = access_needs['residents'] / 60

Load rebuilding regions
Load access needs


In [4]:
import geopandas as gpd
from snman import *
from snman.rebuilding import *

HLAs = multi_rebuild_regions(
    G, rebuilding_regions_gdf, access_needs,
    save_steps_path=os.path.join(export_path, 'steps'),
    save_steps_scaling_factor=2,
    gravity_iterations=10,
)
HLAs = list(filter(lambda HLA: HLA is not None, HLAs))

truncating street graph
done truncating
GIVEN LANES
REMOVE PARKING
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
REMOVE CAR LANES
iteration 1
removed (31, 7, '7-31-0-0')
iteration 2
removed (11, 32, '11-32-0-0')
iteration 3
removed (11, 12, '11-12-0-0')
iteration 4
removed (30, 41, '30-41-0-0')
iteration 5
removed (13, 5, '5-13-0-0')
iteration 6
removed (13, 14, '13-14-0-0')
iteration 7
removed (19, 2, '2-19-0-0')
iteration 8
removed (23, 3, '3-23-0-0')
iteration 9
fixed (24, 17, '17-24-0-0')
iteration 10
fixed (2, 19, '2-19-0-0')
iteration 11
removed (19, 24, '19-24-0-0')
iteration 12
fixed (3, 23, '3-23-0-0')
iteration 13
removed (25, 21, '21-25-0-0')
ite

In [5]:
import networkx as nx

Hs,Ls,As = zip(*HLAs)
A = nx.compose_all(As)
L = nx.compose_all(Ls)

snman.io.export_HLA(os.path.join(export_path, 'steps'), '_debug', L=L, A=A)

In [6]:
print('Reorder lanes')
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION)
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

print('Normalize cycling infrastructure')
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION)
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION_AFTER)

snman.street_graph.organize_edge_directions(G)

Reorder lanes
Normalize cycling infrastructure


In [7]:
print('Add elevation')
G = oxc.elevation.add_node_elevations_raster(
    G,
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'ch_dhm_25', 'ch_dhm_2056.tif'),
    raster_crs=2056,
    #os.path.join(inputs_path, 'copernicus_dem', 'Copernicus_DSM_10_N41_00_W088_00', 'DEM', 'Copernicus_DSM_10_N41_00_W088_00_DEM.tif'),
    #raster_crs=4326, #Chicago
    graph_crs=CRS_internal,
    cpus=1
)
G = oxc.elevation.add_edge_grades(G, add_absolute=False)

Add elevation
                  x             y
osmid                            
0      2.682650e+06  1.250982e+06
1      2.681991e+06  1.251791e+06
2      2.682359e+06  1.251703e+06
3      2.682564e+06  1.251651e+06
4      2.682953e+06  1.251556e+06
5      2.682753e+06  1.251031e+06
6      2.682881e+06  1.251234e+06
7      2.682476e+06  1.251380e+06
8      2.682424e+06  1.251320e+06
9      2.682749e+06  1.251487e+06
10     2.682593e+06  1.251526e+06
11     2.682534e+06  1.251350e+06
12     2.682751e+06  1.251226e+06
13     2.682687e+06  1.251083e+06
14     2.682615e+06  1.251034e+06
15     2.682618e+06  1.251638e+06
17     2.682281e+06  1.251506e+06
18     2.682043e+06  1.251745e+06
19     2.682348e+06  1.251657e+06
20     2.682895e+06  1.251403e+06
21     2.682336e+06  1.251437e+06
22     2.682990e+06  1.251536e+06
23     2.682543e+06  1.251596e+06
24     2.682316e+06  1.251566e+06
25     2.682448e+06  1.251430e+06
26     2.681929e+06  1.251826e+06
27     2.682806e+06  1.251437e+06


In [8]:
if 0:

    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION)
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width', lanetypes={LANETYPE_CYCLING_LANE})
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width')
    snman.street_graph.lane_graph_to_street_graph(
        G, L,
        KEY_LANES_DESCRIPTION
    )
    
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width', lanetypes={LANETYPE_CYCLING_LANE})
    snman.rebuilding._adjust_width_of_lanes(L, None, G, 'width')
    snman.street_graph.lane_graph_to_street_graph(
        G, L,
        KEY_LANES_DESCRIPTION_AFTER
    )

In [9]:
snman.space_allocation.generate_lane_stats(G, lanes_attribute=KEY_LANES_DESCRIPTION)
snman.space_allocation.generate_lane_stats(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

In [10]:
if 1:
    print('export G')
    snman.io.export_street_graph(
        G,
        os.path.join(export_path, 'G_edges.gpkg'),
        os.path.join(export_path, 'G_nodes.gpkg')
    )

if 1:
    print('export L before')
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION)
    snman.io.export_lane_geometries(
        L,
        os.path.join(export_path, 'lane_geometries_edges.gpkg'),
        os.path.join(export_path, 'lane_geometries_nodes.gpkg'),
        scaling=2
    )

if 1:
    print('export L after')
    L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)
    snman.io.export_lane_geometries(
        L,
        os.path.join(export_path, 'lane_geometries_rebuilt_edges.gpkg'),
        os.path.join(export_path, 'lane_geometries_rebuilt_nodes.gpkg'),
        scaling=2
    )

export G
export L before
export L after


In [11]:
if 0:
    print('Load street graph')
    G = snman.io.load_street_graph(
        os.path.join(export_path, 'G_edges.gpkg'),
        os.path.join(export_path, 'G_nodes.gpkg'),
        crs=CRS_internal
    )
    
    street_graph.organize_edge_directions(G)